## Analítica Computacional para la Toma de Decisiones 2024

### Clases 11-12: Introducción a redes neuronales

- Perceptrón Multicapa (MultiLayer perceptron - MLP)
- Redes densas
- Funciones de activación y pérdida

Iniciamos verificando que la versión de python sea por lo menos 3.7

In [ ]:
import sys
assert sys.version_info >= (3, 7)

Verificamos ahora que la versión de Scikit learn sea por lo menos 1.0.1

In [ ]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

Y que la versión de Tensorflow sea por lo menos 2.8.0

In [ ]:
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

Entrenamos un perceptrón usando Scikit Learn sobre los datos Iris para clasificar si una observación corresponde a la especie Setosa o no

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target == 0)  # Iris setosa


Exploremos las observaciones (características)

In [ ]:
print(X.shape)
print(X[0:5,])
print(X[-5:150,])

Exploremos las observaciones (etiquetas)

In [ ]:
print(y.shape)
print(y[0:5,])
print(y[-5:150,])

Entrenemos el perceptron usando una semilla para hacer que los experimentos sean repetibles

In [ ]:
perc = Perceptron(random_state=42)
perc.fit(X, y)

Probemos el modelo con un par de datos nuevos

In [ ]:
Xnew = [[2, 0.3], [4, 2]]
ypred = perc.predict(Xnew)  
print(ypred)

Gráfico de las observaciones clasificadas en cada grupo: Setosa o no Setosa

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

a = -perc.coef_[0, 0] / perc.coef_[0, 1]
b = -perc.intercept_ / perc.coef_[0, 1]
axes = [0, 5, 0, 2]
x0, x1 = np.meshgrid(
    np.linspace(axes[0], axes[1], 500).reshape(-1, 1),
    np.linspace(axes[2], axes[3], 200).reshape(-1, 1),
)
X_new = np.c_[x0.ravel(), x1.ravel()]
y_predict = perc.predict(X_new)
zz = y_predict.reshape(x0.shape)
custom_cmap = ListedColormap(['#9898ff', '#fafab0'])

plt.figure(figsize=(7, 3))
plt.plot(X[y == 0, 0], X[y == 0, 1], "bs", label="No setosa")
plt.plot(X[y == 1, 0], X[y == 1, 1], "yo", label="Setosa")
plt.plot([axes[0], axes[1]], [a * axes[0] + b, a * axes[1] + b], "k-",
         linewidth=1)
plt.contourf(x0, x1, zz, cmap=custom_cmap)
plt.xlabel("Longitud del pétalo")
plt.ylabel("Ancho del pétalo")
plt.legend(loc="lower right")
plt.axis(axes)
plt.show()

Ahora utilicemos un perceptrón multicapa, con los datos completos de Iris, dividiendo los datos en entrenamiento, validación y prueba

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

iris = load_iris()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42)



Estandaricemos los datos de entrada y entrenemos un perceptrón multicapa con una capa oculta con 3 neuronas

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
mlp_clf = MLPClassifier(hidden_layer_sizes=[5], max_iter=1000,
                        random_state=42)
pipeline = make_pipeline(StandardScaler(), mlp_clf)
pipeline.fit(X_train, y_train)
accuracy = pipeline.score(X_valid, y_valid)
accuracy

Ahora construyamos el mismo modelo usando Keras y Tensorflow

In [ ]:
import tensorflow as tf

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()


print(iris.data[0:3,])
print(iris.target[0:3,])

y = tf.keras.utils.to_categorical(iris.target, num_classes=3)

print(y[0:3,])
print(y.shape)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    iris.data, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42)

X_train.shape

std_scl = StandardScaler()
std_scl.fit(X_train)

print(X_train[0:3,])
X_train = std_scl.transform(X_train)
print(X_train[0:3,])
X_valid = std_scl.transform(X_valid)
X_test = std_scl.transform(X_test)



In [ ]:
X_train.dtype

In [ ]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()

model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(4,)))
model.add(tf.keras.layers.Dense(5, activation="relu"))
model.add(tf.keras.layers.Dense(3, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
weights, biases = hidden1.get_weights()
weights

In [ ]:
weights.shape

In [ ]:
import pandas as pd
pd.DataFrame(weights)

In [ ]:
biases

In [ ]:
biases.shape

In [ ]:
pd.DataFrame(biases)

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=1000,
                    validation_data=(X_valid, y_valid))

## Datos MNIST fashion
Ahora entrenemos otro modelo de clasificación, esta vez para datos MNIST fashion

In [ ]:
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [ ]:
X_train.shape

In [ ]:
pd.DataFrame(X_train[0,:])

Reescalemos los datos para que cada pixel tenga un valor entre 0 y 1

In [ ]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.

In [ ]:
pd.DataFrame(X_train[0,:])

In [ ]:
plt.imshow(X_train[0], cmap="binary")
plt.axis('off')
plt.show()

In [ ]:
y_train[0]

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

class_names[y_train[0]]

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]])
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

Definimos el modelo con
- 1 capa de entrada
- 1 operación de adelgazamiento (flatten)
- 2 capas densas con función de activación ReLU
- 1 capa de salida con función de activación softmax (clasificación) con tantas neuronas como categorías tenemos etiquetadas (10)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
weights, biases = hidden1.get_weights()
weights.shape

In [ ]:
biases.shape

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))